In [1]:
import tensorflow as tf
tf.enable_eager_execution()

import numpy as np
import os
import time

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1122304/1115394 [==============================] - 2s 2us/step


In [3]:
text = open(path_to_file).read()
len(text)

1115394

In [6]:
print(text[ :200])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you


In [7]:
vocab = sorted(set(text))
print(len(vocab))
print(vocab)

65
['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [8]:
char2idx = {v: k for k, v in enumerate(vocab)}
idx2char = np.array(vocab)

In [11]:
text_as_int = np.array([char2idx[c] for c in text])

In [20]:
seq_length = 100
chunks = tf.data.Dataset.from_tensor_slices(text_as_int).batch(seq_length + 1, drop_remainder = True)

for item in chunks.take(5):
    print(repr("".join(idx2char[item.numpy()])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [21]:
def split_input_target(chunk):
    input_text = chunk[ :-1]
    target_text = chunk[1: ]
    return input_text, target_text

dataset = chunks.map(split_input_target)

In [24]:
for input_example, target_example in dataset.take(1):
    print(repr("".join(idx2char[input_example.numpy()])))
    print(repr("".join(idx2char[target_example.numpy()])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [25]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder = True)

In [26]:
class Model(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, units):
        super(Model, self).__init__()
        self.units = units
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        
        self.gru = tf.keras.layers.GRU(self.units,
                                       return_sequences = True,
                                       recurrent_activation = 'sigmoid',
                                       recurrent_initializer = 'glorot_uniform',
                                       stateful = True)
        self.fc = tf.keras.layers.Dense(vocab_size)
    
    def call(self, x):
        embedding = self.embedding(x)
        output = self.gru(embedding)
        prediction = self.fc(output)
        return prediction

In [27]:
vocab_size = len(vocab)
embedding_dim = 256
units = 1024

model = Model(vocab_size, embedding_dim, units)

In [28]:
optimizer = tf.train.AdamOptimizer()

def loss_function(real, preds):
    return tf.losses.sparse_softmax_cross_entropy(labels = real, logits = preds)

In [30]:
model.build(tf.TensorShape([BATCH_SIZE, seq_length]))
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  16640     
_________________________________________________________________
gru (GRU)                    multiple                  3935232   
_________________________________________________________________
dense (Dense)                multiple                  66625     
Total params: 4,018,497
Trainable params: 4,018,497
Non-trainable params: 0
_________________________________________________________________


In [31]:
EPOCHS = 5

In [32]:
for epoch in range(EPOCHS):
    
    start = time.time()

    hidden = model.reset_states()
    for (batch, (inp, target)) in enumerate(dataset):
        with tf.GradientTape() as tape:
            predictions = model(inp)
            loss = loss_function(target, predictions)
        grads = tape.gradient(loss, model.variables)
        optimizer.apply_gradients(zip(grads, model.variables))
        
        if batch % 100 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1, batch, loss))
    
    print ('Epoch {} Loss {:.4f}'.format(epoch+1, loss))
    print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))
    

Epoch 1 Batch 0 Loss 4.1755
Epoch 1 Batch 100 Loss 2.3384
Epoch 1 Loss 2.1026
Time taken for 1 epoch 613.64439868927 sec

Epoch 2 Batch 0 Loss 2.1100


KeyboardInterrupt: 